In [1]:
import pandas as pd
import sys
sys.path.append("..")
from src.Eval import eval_llm

import os
import io
from contextlib import redirect_stdout


In [11]:
import sys
sys.path.append("..")
from src.Eval import eval_llm

import os
import io
from contextlib import redirect_stdout

def parse_filename(filename):
    parts = filename.split("_")
    dataset = parts[0]
    k_shot = parts[1].split(".")[0]
    k = int(k_shot.replace("shot", ""))
    return dataset, k

def silent_eval_llm(fpath):
    with io.StringIO() as buf, redirect_stdout(buf):
        # Call the actual eval_llm function
        return eval_llm(fpath)
    
def aggregated_results_llm(
    results_dir,
    ent_types=["overall", "Artist", "WoA"],
    eval_schemas=["strict", "exact", "ent_type"],
    metrics=["f1", "f1_macro", "precision", "precision_macro", "recall", "recall_macro", "missed", "spurious", "incorrect"],
    datasets=["dataset1", "dataset2", "dataset3", "dataset4"],
    ks=[0,5,15,25,35,45]
):
    results = {}
    for metric in metrics:
        results[metric] = {}
        for schema in eval_schemas:
            results[metric][schema] = {}
            for ent_type in ent_types:
                results[metric][schema][ent_type] = {}
                for model in [m for m in os.listdir(results_dir) if m != "archive"]:
                    results[metric][schema][ent_type][model] = {}
                    for k in ks:
                        results[metric][schema][ent_type][model][k] = []

    model_dirs = [
        os.path.join(results_dir, d)
        for d in os.listdir(results_dir)
        if os.path.isdir(os.path.join(results_dir, d)) and d != "archive"
    ]

    for model_dir in model_dirs:
        model = model_dir.split(os.sep)[-1]
        files = [
            os.path.join(model_dir, f)
            for f in os.listdir(model_dir)
            if os.path.isfile(os.path.join(model_dir, f))
        ]

        for fpath in files:
            filename = os.path.basename(fpath)
            dataset, k = parse_filename(filename)

            if dataset in datasets and k in ks:
                predictions = silent_eval_llm(fpath)

                for ent_type in ent_types:
                    for schema in eval_schemas:
                        for metric in metrics:
                            key = f"{ent_type}_{schema}_{metric}"
                            if key in predictions.keys():
                                results[metric][schema][ent_type][model][k].append(predictions[key])
    return results

results = aggregated_results_llm("../output/reddit+shsyt/")
#results_tfidf = aggregated_results_llm("../output/tfidf_sampling")



2024-09-02 12:36:58 root INFO: Imported 660 predictions for 660 true examples
2024-09-02 12:36:58 root INFO: Imported 660 predictions for 660 true examples


In [12]:
results

{'f1': {'strict': {'overall': {'dataset1': {0: [],
     5: [],
     15: [],
     25: [],
     35: [],
     45: []},
    'llama3.1-70b': {0: [], 5: [], 15: [], 25: [], 35: [], 45: []},
    'llama3.1-8b': {0: [], 5: [], 15: [], 25: [], 35: [], 45: []}},
   'Artist': {'dataset1': {0: [], 5: [], 15: [], 25: [], 35: [], 45: []},
    'llama3.1-70b': {0: [0.783641160949868],
     5: [],
     15: [],
     25: [],
     35: [],
     45: []},
    'llama3.1-8b': {0: [0.8048452220726784],
     5: [],
     15: [],
     25: [],
     35: [],
     45: []}},
   'WoA': {'dataset1': {0: [], 5: [], 15: [], 25: [], 35: [], 45: []},
    'llama3.1-70b': {0: [0.6446991404011462],
     5: [],
     15: [],
     25: [],
     35: [],
     45: []},
    'llama3.1-8b': {0: [0.6885758998435054],
     5: [],
     15: [],
     25: [],
     35: [],
     45: []}}},
  'exact': {'overall': {'dataset1': {0: [],
     5: [],
     15: [],
     25: [],
     35: [],
     45: []},
    'llama3.1-70b': {0: [], 5: [], 15: [], 25: [],

In [14]:
import pandas as pd
import numpy as np

def results_to_dataframe(aggregated_results, metric, agg_func='mean'):
    data = []

    for schema, schema_dict in aggregated_results[metric].items():
        for ent_type, ent_type_dict in schema_dict.items():
            for model, model_dict in ent_type_dict.items():
                for k_shot, values in model_dict.items():
                    if values:
                        if agg_func == 'mean':
                            agg_value = np.mean(values)
                        elif agg_func == 'sum':
                            agg_value = np.sum(values)
                        else:
                            raise ValueError("Invalid aggregation function. Use 'mean' or 'sum'.")
                        
                        data.append([schema, ent_type, model, k_shot, agg_value])

    df = pd.DataFrame(data, columns=['Schema', 'Entity Type', 'Model', 'k', 'Value'])
    df.Model = df.Model.str.replace("mistral", "Mistral-7B").str.replace("mixtral", "Mixtral-8x22B").str.replace("gpt-3.5-turbo-0125", "GPT-3.5-Turbo")
    return df.set_index(["Schema", "Entity Type", "Model", "k"])



# Precision

In [17]:
_results = results_to_dataframe(results, "precision", "mean").reset_index()
_results[(_results.Schema == "strict") & (_results.k == 0)].drop(["Schema", "k"], axis=1)


,Entity Type,Model,Value
0,Artist,llama3.1-70b,0.783641
1,Artist,llama3.1-8b,0.821429
2,WoA,llama3.1-70b,0.589005
3,WoA,llama3.1-8b,0.681115


# Recall

In [20]:
_results = results_to_dataframe(results, "recall", "mean").reset_index()
_results[(_results.Schema == "strict") & (_results.k == 0)].drop(["Schema", "k"], axis=1)



,Entity Type,Model,Value
0,Artist,llama3.1-70b,0.783641
1,Artist,llama3.1-8b,0.788918
2,WoA,llama3.1-70b,0.712025
3,WoA,llama3.1-8b,0.696203


# F1

In [21]:
results_to_dataframe(results, "f1_macro", "mean")


Value
Schema   Entity Type Model        k          
strict   overall     llama3.1-70b 0  0.714170
                     llama3.1-8b  0  0.746711
exact    overall     llama3.1-70b 0  0.747152
                     llama3.1-8b  0  0.763299
ent_type overall     llama3.1-70b 0  0.774868
                     llama3.1-8b  0  0.790561

In [25]:
results_to_dataframe(results, "f1", "mean")


Value
Schema   Entity Type Model        k          
strict   Artist      llama3.1-70b 0  0.783641
                     llama3.1-8b  0  0.804845
         WoA         llama3.1-70b 0  0.644699
                     llama3.1-8b  0  0.688576
exact    Artist      llama3.1-70b 0  0.849604
                     llama3.1-8b  0  0.831763
         WoA         llama3.1-70b 0  0.644699
                     llama3.1-8b  0  0.694836
ent_type Artist      llama3.1-70b 0  0.804749
                     llama3.1-8b  0  0.823688
         WoA         llama3.1-70b 0  0.744986
                     llama3.1-8b  0  0.757433